### To create a df containing all the transaction data

In [270]:
import csv
import os
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import re
import pandas as pd
import numpy as np
import datetime

In [2]:
url = 'https://fantasy.nfl.com/league/6787729/transactions'
page = urlopen(url)
html = page.read()
page.close()
soup1 = bs(html, 'html.parser') 

In [3]:
def soup_cleaner(df):
    trans = []
    row_marker = 0
    for row in df.find_all('tr'):
        #print(row_marker)
        column_marker = 0
        columns = row.find_all('td')
        col_data = []
        for column in columns:
            col_data.append(column.get_text())
            column_marker +=1
        trans.append(col_data)
        row_marker += 1

    return(trans)

In [4]:
url = 'https://fantasy.nfl.com/league/6787729/transactions?offset=61'
page = urlopen(url)
html = page.read()
page.close()
canned = bs(html, 'html.parser') 
soup = canned.find_all('table')[0]
soup;

In [4]:
def CanadasChunkySoup(url, offset,bowlsize):
    counter = 0
    off_num = 0
    while bowlsize >= off_num:
        if counter == 0:
            page = urlopen(url)
        elif counter > 0:
            page = urlopen(url+'?offset='+str(off_num))
        html = page.read()
        page.close()
        canned = bs(html, 'html.parser') 
        soup = canned.find_all('table')[0]
        cleansoup = soup_cleaner(soup)
        if counter == 0: 
            soupbowl = cleansoup
        if counter > 0:
            soupbowl = soupbowl + cleansoup
        counter += 1
        off_num = (offset*counter)+1
    soupbowl = pd.DataFrame(soupbowl)
    return soupbowl

In [341]:
%%time
# the last table is 1801 away
tables = CanadasChunkySoup('https://canada.fantasy.nfl.com/league/6787729/transactions', 20,1801)
tables

CPU times: user 5.67 s, sys: 370 ms, total: 6.04 s
Wall time: 45.4 s


,0,1,2,3,4,5,6
0,None,None,None,None,None,None,None
1,"Jan 16, 3:49pm",,LM,Jake changed Viewable to Public from 'No' to '...,Jake,None,None
2,"Dec 27, 8:36am",16,Lineup,James Robinson RB - JAX Q,RB,BN,Sam via Mobile Device
3,"Dec 27, 8:36am",16,Lineup,Antonio Gibson RB - WAS,BN,RB,Sam via Mobile Device
4,"Dec 27, 7:09am",16,Lineup,Brandin Cooks WR - HOU,BN,WR,Matt
...,...,...,...,...,...,...,...
2122,"Sep 7, 1:58pm",,LM,Jake changed Waiver Budget to '100',Jake,None,None
2123,"Sep 7, 1:58pm",,LM,Jake changed Waiver Type from 'Resets to Inver...,Jake,None,None
2124,"Sep 7, 1:55pm",,LM,"Jake changed Draft Time to 'Sep 8, 2020 5:30pm...",Jake,None,None
2125,"Sep 7, 1:55pm",,LM,Jake changed Draft Type from 'offline' to 'live',Jake,None,None


#### Now that we have the transaction data, lets clean a bit

Data cleaning issues
* Players with hyphens in their name make it difficult to string split
* Defenses are formatted differently
* Trades can have multiple players and so will string split differently
* There are status symbols that have to be cleaned from the ends of the player names
* Transaction Costs for adds
* The listed team is the player's NFL team at the end of the year, no way currently to deal with if the player switched NFL teams in season
* The Jordan Howard Problem, where the player has no team

So I think we do the following:
1. Remove Trades to be processed separately
2. Split Defeneses to be processed separately
3. Format Players to have their names, positions, and team in separate columns, dropping status symbols for all three DFs
4. Rejoin Trades, Defenses, and the rest 
5. Process Transaction Costs from Waiver Adds

In [342]:
# lets give some column names and drop any rows that have the LM designation or NA
tables.columns = ['Date','Week','TransType','Player','From','To','By']
tables = tables[tables.TransType!='LM']
tables = tables.dropna()
tables['TransType'].value_counts()

Lineup    1446
Add        282
Drop       277
Trade       14
Name: TransType, dtype: int64

In [343]:
jHow = tables[tables.Player.str.contains('Jordan Howard')]
jHow

,Date,Week,TransType,Player,From,To,By
1984,"Sep 16, 1:02am",2,Drop,Jordan Howard RB,Fantasy Football Team,Waivers,Liam
2102,"Sep 8, 11:08pm",1,Lineup,Jordan Howard RB,R/W/T,BN,Liam via Mobile Device
2105,"Sep 8, 10:50pm",1,Lineup,Jordan Howard RB,BN,R/W/T,Liam via Mobile Device


In [ ]:
tables[(tables.TransType == 'Add') & (tables.From == 'Waivers')];

Split off Trades

In [345]:
trades = pd.DataFrame(tables[tables['TransType'] == 'Trade'],copy = True)
trades;

Split off defenses

In [346]:
DEF = tables[tables.Player.str.contains('DEF')].copy()
DEF;

Isolate everyone else

In [347]:
Normal = pd.DataFrame(tables[~((tables['TransType'] == 'Trade') | (tables.Player.str.contains('DEF')))], copy = True)
Normal;

#### First let's clean up the StandardData

In [348]:
def table_cleaner(df):
    df['Team'] = df.Player.str.split(pat = " - ", expand = False).str[1].str.split().str[0]
    df['Position'] = df.Player.str.split(pat = " - ", expand = False).str[0].str.split().str[-1]
    df['Player'] = df.Player.str.split(pat = " - ", expand = False).str[0].str.split().str[:-1].str.join(" ")
    return df

In [349]:
Normal = table_cleaner(Normal)
Normal

,Date,Week,TransType,Player,From,To,By,Team,Position
2,"Dec 27, 8:36am",16,Lineup,James Robinson,RB,BN,Sam via Mobile Device,JAX,RB
3,"Dec 27, 8:36am",16,Lineup,Antonio Gibson,BN,RB,Sam via Mobile Device,WAS,RB
4,"Dec 27, 7:09am",16,Lineup,Brandin Cooks,BN,WR,Matt,HOU,WR
5,"Dec 27, 7:09am",16,Lineup,Keenan Allen,WR,BN,Matt,LAC,WR
6,"Dec 26, 9:43pm",16,Lineup,Tyler Boyd,WR,BN,f,CIN,WR
...,...,...,...,...,...,...,...,...,...
2112,"Sep 8, 6:49pm",1,Lineup,Noah Fant,BN,R/W/T,f,DEN,TE
2113,"Sep 8, 6:42pm",1,Lineup,Melvin Gordon,RB,R/W/T,Jess,DEN,RB
2114,"Sep 8, 6:42pm",1,Lineup,Mark Ingram,R/W/T,RB,Jess,BAL,RB
2115,"Sep 8, 6:40pm",1,Lineup,Damien Harris,BN,RES,Sam via Mobile Device,NE,RB


#### Now let's clean up the DEF and trades

In [350]:
TeamDict = dict({
 'ARI':'Arizona Cardinals',
 'BAL':'Baltimore Ravens',
 'BUF':'Buffalo Bills',
 'CAR':'Carolina Panthers',
 'CHI':'Chicago Bears',
 'CLE':'Cleveland Browns',
 'DEN':'Denver Broncos',
 'GB':'Green Bay Packers',
 'IND':'Indianapolis Colts',
 'KC':'Kansas City Chiefs',
 'LAC':'Los Angeles Chargers',
 'LAR':'Los Angeles Rams',
 'MIA':'Miami Dolphins',
 'MIN':'Minnesota Vikings',
 'NE':'New England Patriots',
 'NO':'New Orleans Saints',
 'NYG':'New York Giants',
 'PHI':'Philadelphia Eagles',
 'SF':'San Francisco 49ers',
 'SEA':'Seattle Seahawks',
 'TB':'Tampa Bay Buccaneers',
 'WAS':'Washington Football Team'})
TeamDict_Rev = {value:key for (key,value) in TeamDict.items()}

In [ ]:
# let's clean up the DEF
# split them via a similar method
DEF['Position'] = DEF.Player.str.split(pat = " ", expand = False).str[-2]
DEF['Team'] = DEF.Player.str.split(pat = " ", expand = False).str[:-2].str.join(" ")
# remap the team names to be consistent
DEF['Team'] = DEF['Team'].map(TeamDict_Rev)
DEF

To process the trades, if there are multiple players in a cell, all the other information has be to duplicated and the player split off into their own row

In [353]:
trades

,Date,Week,TransType,Player,From,To,By
313,"Nov 29, 9:40am",12,Trade,Tyreek Hill WR - KC,Eatin W's,The Wet Market Pangolins,Jake
314,"Nov 29, 9:40am",12,Trade,Aaron Jones RB - GB Q,The Wet Market Pangolins,Eatin W's,Jake
925,"Nov 1, 12:14am",8,Trade,Clyde Edwards-Helaire RB - KC,Do it for George,The Wet Market Pangolins,Jake
926,"Nov 1, 12:14am",8,Trade,A.J. Brown WR - TEN Q,The Wet Market Pangolins,Do it for George,Jake
1111,"Oct 24, 12:39am",7,Trade,D'Andre Swift RB - DET,ganggreen215,Gronk's New Groove,Sam
1112,"Oct 24, 12:39am",7,Trade,Terry McLaurin WR - WAS,Gronk's New Groove,ganggreen215,Sam
1581,"Oct 3, 5:21pm",4,Trade,Jonathan Taylor RB - IND,ganggreen215,Eatin W's,Jake
1582,"Oct 3, 5:21pm",4,Trade,Jarvis Landry WR - CLE James Conner RB - PIT,Eatin W's,ganggreen215,Jake
1623,"Oct 2, 12:25am",4,Trade,Ezekiel Elliott RB - DAL,Eatin W's,Triple Crown,Nolan
1624,"Oct 2, 12:25am",4,Trade,Mike Evans WR - TB Kenyan Drake RB - ARI,Triple Crown,Eatin W's,Nolan


In [354]:
def trade_cleaner(df):
    # find trades with multiple players and split from single player trades
    # for multi player trades, duplicate the row
    multi = df.loc[df[df['Player'].str.count(" - ") > 1].index.repeat(2)]
    singles = df[df['Player'].str.count(" - ") == 1]
    for i in set(multi.index):
        # rewrite the top row with one player, and the bottom row with the other, NOTE: this only works with two players max
        multi.loc[i,'Player'].iloc[0] = multi.loc[i,'Player'].iloc[0].split(" - ")[0] +" - "+ multi.loc[i,'Player'].iloc[0].split(" - ")[1].split(" ")[0]
        multi.loc[i,'Player'].iloc[1] = " ".join(multi.loc[i,'Player'].iloc[1].split(" - ")[1].split(" ")[-3:]) +" - "+ multi.loc[i,'Player'].iloc[1].split(" - ")[2]
    df = pd.concat([singles,multi],axis = 0).reset_index(drop = True)
    df = df.sort_values(by='Date')
    return(df)

In [355]:
trades = trade_cleaner(trades)
trades

,Date,Week,TransType,Player,From,To,By
2,"Nov 1, 12:14am",8,Trade,Clyde Edwards-Helaire RB - KC,Do it for George,The Wet Market Pangolins,Jake
3,"Nov 1, 12:14am",8,Trade,A.J. Brown WR - TEN Q,The Wet Market Pangolins,Do it for George,Jake
0,"Nov 29, 9:40am",12,Trade,Tyreek Hill WR - KC,Eatin W's,The Wet Market Pangolins,Jake
1,"Nov 29, 9:40am",12,Trade,Aaron Jones RB - GB Q,The Wet Market Pangolins,Eatin W's,Jake
7,"Oct 2, 12:25am",4,Trade,Ezekiel Elliott RB - DAL,Eatin W's,Triple Crown,Nolan
13,"Oct 2, 12:25am",4,Trade,Kenyan Drake RB - ARI,Triple Crown,Eatin W's,Nolan
12,"Oct 2, 12:25am",4,Trade,Mike Evans WR - TB,Triple Crown,Eatin W's,Nolan
4,"Oct 24, 12:39am",7,Trade,D'Andre Swift RB - DET,ganggreen215,Gronk's New Groove,Sam
5,"Oct 24, 12:39am",7,Trade,Terry McLaurin WR - WAS,Gronk's New Groove,ganggreen215,Sam
6,"Oct 3, 5:21pm",4,Trade,Jonathan Taylor RB - IND,ganggreen215,Eatin W's,Jake


Now, apply the necessary cleaning steps

In [356]:
trades = table_cleaner(trades)
trades

,Date,Week,TransType,Player,From,To,By,Team,Position
2,"Nov 1, 12:14am",8,Trade,Clyde Edwards-Helaire,Do it for George,The Wet Market Pangolins,Jake,KC,RB
3,"Nov 1, 12:14am",8,Trade,A.J. Brown,The Wet Market Pangolins,Do it for George,Jake,TEN,WR
0,"Nov 29, 9:40am",12,Trade,Tyreek Hill,Eatin W's,The Wet Market Pangolins,Jake,KC,WR
1,"Nov 29, 9:40am",12,Trade,Aaron Jones,The Wet Market Pangolins,Eatin W's,Jake,GB,RB
7,"Oct 2, 12:25am",4,Trade,Ezekiel Elliott,Eatin W's,Triple Crown,Nolan,DAL,RB
13,"Oct 2, 12:25am",4,Trade,Kenyan Drake,Triple Crown,Eatin W's,Nolan,ARI,RB
12,"Oct 2, 12:25am",4,Trade,Mike Evans,Triple Crown,Eatin W's,Nolan,TB,WR
4,"Oct 24, 12:39am",7,Trade,D'Andre Swift,ganggreen215,Gronk's New Groove,Sam,DET,RB
5,"Oct 24, 12:39am",7,Trade,Terry McLaurin,Gronk's New Groove,ganggreen215,Sam,WAS,WR
6,"Oct 3, 5:21pm",4,Trade,Jonathan Taylor,ganggreen215,Eatin W's,Jake,IND,RB


Now rejoin all the tables, reset the index, make the columns pretty, resort by date, and deal with the fact that Jordan Howard doesn't have a team

In [372]:
trans = pd.concat([Normal,DEF,trades], axis = 0, ignore_index = True)
trans = trans[['Date', 'Week', 'TransType', 'Player', 'Position', 'Team', 'From', 'To', 'By', ]]
trans['Date'] = trans['Date'].apply(lambda x: datetime.datetime.strptime(('2020 '+x),'%Y %b %d, %I:%M%p'))
trans = trans.sort_values(by='Date').reset_index(drop = True)
trans['Player'] = trans['Player'].str.strip()
trans['Team'] = trans['Team'].str.strip()
trans['Position'] = trans['Position'].str.strip()
trans.loc[trans['Team'].isna(),'Team'] = 'No Team'
trans

,Date,Week,TransType,Player,Position,Team,From,To,By
0,2020-09-08 18:39:00,1,Lineup,Ryquell Armstead,RB,JAX,BN,RES,Graham
1,2020-09-08 18:40:00,1,Lineup,Damien Harris,RB,NE,BN,RES,Sam via Mobile Device
2,2020-09-08 18:42:00,1,Lineup,Mark Ingram,RB,BAL,R/W/T,RB,Jess
3,2020-09-08 18:42:00,1,Lineup,Melvin Gordon,RB,DEN,RB,R/W/T,Jess
4,2020-09-08 18:49:00,1,Lineup,Noah Fant,TE,DEN,BN,R/W/T,f
...,...,...,...,...,...,...,...,...,...
2018,2020-12-26 21:43:00,16,Lineup,Tyler Boyd,WR,CIN,WR,BN,f
2019,2020-12-27 07:09:00,16,Lineup,Keenan Allen,WR,LAC,WR,BN,Matt
2020,2020-12-27 07:09:00,16,Lineup,Brandin Cooks,WR,HOU,BN,WR,Matt
2021,2020-12-27 08:36:00,16,Lineup,Antonio Gibson,RB,WAS,BN,RB,Sam via Mobile Device


In [373]:
pd.set_option('display.max_rows', 20)
trans['TransType'].value_counts()

Lineup    1446
Add        282
Drop       277
Trade       18
Name: TransType, dtype: int64

#### Now lets deal with Transaction Costs from the waiver wire

In [374]:
trans[(trans.TransType == 'Add') & (trans.From == 'Waivers')]

,Date,Week,TransType,Player,Position,Team,From,To,By
23,2020-09-10 00:13:00,1,Add,Chris Boswell,K,PIT,Waivers,Eatin W's (0 pts),Graham
24,2020-09-10 00:13:00,1,Add,James Robinson,RB,JAX,Waivers,Gronk's New Groove (1 pts),Sam
25,2020-09-10 00:13:00,1,Add,Philadelphia Eagles DEF,DEF,PHI,Waivers,Eatin W's (1 pts),Graham
124,2020-09-16 01:02:00,2,Add,T.J. Hockenson,TE,DET,Waivers,Gronk's New Groove (1 pts),Sam
126,2020-09-16 01:02:00,2,Add,Russell Gage,WR,ATL,Waivers,Gronk's New Groove (1 pts),Sam
...,...,...,...,...,...,...,...,...,...
1929,2020-12-16 01:42:00,15,Add,Buffalo Bills DEF,DEF,BUF,Waivers,Fantasy Football Team (2 pts),Liam
1930,2020-12-16 01:42:00,15,Add,T.Y. Hilton,WR,IND,Waivers,ganggreen215 (10 pts),Kevin
1931,2020-12-16 01:42:00,15,Add,Cam Akers,RB,LA,Waivers,ganggreen215 (10 pts),Kevin
1984,2020-12-23 00:45:00,16,Add,Le'Veon Bell,RB,KC,Waivers,ganggreen215 (10 pts),Kevin


In [375]:
trans['TransCosts'] = trans['To'].str.split("(").str[1].str.split(")").str[0]
trans['To'] = trans['To'].str.split("(").str[0].str.strip()
trans

,Date,Week,TransType,Player,Position,Team,From,To,By,TransCosts
0,2020-09-08 18:39:00,1,Lineup,Ryquell Armstead,RB,JAX,BN,RES,Graham,NaN
1,2020-09-08 18:40:00,1,Lineup,Damien Harris,RB,NE,BN,RES,Sam via Mobile Device,NaN
2,2020-09-08 18:42:00,1,Lineup,Mark Ingram,RB,BAL,R/W/T,RB,Jess,NaN
3,2020-09-08 18:42:00,1,Lineup,Melvin Gordon,RB,DEN,RB,R/W/T,Jess,NaN
4,2020-09-08 18:49:00,1,Lineup,Noah Fant,TE,DEN,BN,R/W/T,f,NaN
...,...,...,...,...,...,...,...,...,...,...
2018,2020-12-26 21:43:00,16,Lineup,Tyler Boyd,WR,CIN,WR,BN,f,NaN
2019,2020-12-27 07:09:00,16,Lineup,Keenan Allen,WR,LAC,WR,BN,Matt,NaN
2020,2020-12-27 07:09:00,16,Lineup,Brandin Cooks,WR,HOU,BN,WR,Matt,NaN
2021,2020-12-27 08:36:00,16,Lineup,Antonio Gibson,RB,WAS,BN,RB,Sam via Mobile Device,NaN


In [376]:
trans['To'].value_counts()

BN                          594
Waivers                     277
WR                          225
R/W/T                       216
RB                          176
TE                           60
Gronk's New Groove           57
DEF                          51
Do it for George             47
RES                          44
K                            41
QB                           39
Fantasy Football Team        38
Eatin W's                    34
ganggreen215                 33
Triple Crown                 29
The Wet Market Pangolins     24
The Longest Yard             16
Tight Ends Spread Sheets     13
KaepernicksRevenge            9
Name: To, dtype: int64

In [ ]:
trans.to_csv(index = )